<a href="https://colab.research.google.com/github/Sambradshaw19011/CSE-450/blob/main/raisins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import pandas as pd
import sklearn.model_selection as ms
import sklearn.ensemble as en
import sklearn.metrics as met
import matplotlib.pyplot as plt

campaign = pd.read_csv('https://byui-cse.github.io/cse450-course/ice/raisins/data/raisin-training.csv')

In [30]:
campaign.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,97026,455.971591,273.053810,0.800869,99561,0.671205,1212.667,Kecimen
1,65253,418.997887,205.756185,0.871122,69700,0.666255,1075.404,Besni
2,113029,558.516156,265.284203,0.879996,116783,0.662092,1419.577,Besni
3,76792,338.857545,291.359202,0.510584,78842,0.772322,1042.770,Kecimen
4,72219,376.650492,249.529454,0.749065,74373,0.777795,1050.221,Kecimen


In [33]:
X = pd.get_dummies(campaign.drop(columns=["Class"]), drop_first=True)
y = campaign["Class"]

X_train, X_test, y_train, y_test = ms.train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

clf = en.RandomForestClassifier(
    n_estimators=600,          # Number of trees in the forest (more trees = usually better performance but slower)

    max_depth=15,              # Maximum depth of each tree (limits complexity to reduce overfitting)

    min_samples_leaf=10,       # Minimum samples required in a leaf node (higher = smoother trees, less overfitting)

    random_state=42,           # Makes results reproducible (same randomness every run)

    class_weight="balanced",  # Penalizes mistakes on "yes" more heavily (helps catch more "yes" cases / raises recall)

    max_features="sqrt",       # Number of features each split can consider (sqrt is common; adds randomness, reduces overfitting)

    bootstrap=True,            # Uses bootstrapped samples (random sampling with replacement) for each tree (default RF behavior)

    n_jobs=-1                  # Uses all CPU cores to train faster
)


clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print(met.classification_report(y_test, pred))
print(met.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

       Besni       0.86      0.81      0.83       104
     Kecimen       0.82      0.87      0.84       106

    accuracy                           0.84       210
   macro avg       0.84      0.84      0.84       210
weighted avg       0.84      0.84      0.84       210

[[84 20]
 [14 92]]


In [35]:
holdout = pd.read_csv('https://byui-cse.github.io/cse450-course/ice/raisins/data/raisin-holdout.csv')


train_cols = X.columns

X_holdout = pd.get_dummies(
    holdout.drop(columns=["Class"], errors="ignore"),
    drop_first=True
)

X_holdout = X_holdout.reindex(columns=train_cols, fill_value=0)

holdout_pred = clf.predict(X_holdout)

pd.DataFrame({"Class": holdout_pred}).to_csv(
    "SAM-BRADSHAW-raisins-predictions.csv",
    index=False
)


In [ ]:
plt.figure(figsize=(9, 5))
plt.scatter(campaign["Area"], campaign["Class"], alpha=0.2, s=10)
plt.title("Area vs Class")
plt.xlabel("Area")
plt.ylabel("Class")
plt.tight_layout()
plt.show()